In [1]:
import random, gc, os, pickle, csv

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-ori/OML-order1-id4-2023-04-25_22-37-58.874941.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-ori/OML-order1-id4-2023-04-25_22-37-58.874970_memory.pickle"


use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*10,
    "meta_lr": 3e-5,
    "model": "roberta", # changed from bert
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600,
    "pln": "1fc"
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
    learner.memory = pickle.load(f)

2023-09-14 10:07:57,009 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-09-14 10:07:57,014 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-09-14 10:07:58,370 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [7]:
def evaluate(dataloader, updates, mini_batch_size):
    learner.rln.eval()
    learner.pln.train()
    
    support_set = []
    for _ in range(updates):
        text, labels = learner.memory.read_batch(batch_size=mini_batch_size)
        support_set.append((text, labels))
    
    with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                              copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):

        # Inner loop
        task_predictions, task_labels = [], []
        support_loss = []
        for text, labels in support_set:
            labels = torch.tensor(labels).to(device)
            input_dict = learner.rln.encode_text(text)
            _repr = learner.rln(input_dict)
            output = fpln(_repr)
            loss = learner.loss_fn(output, labels)
            diffopt.step(loss)
            pred = models.utils.make_prediction(output.detach())
            support_loss.append(loss.item())
            task_predictions.extend(pred.tolist())
            task_labels.extend(labels.tolist())

        acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

        print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                    'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

        all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []

        for text, labels in dataloader:
            labels = torch.tensor(labels).to(device)
            input_dict = learner.rln.encode_text(text)
            with torch.no_grad():
                _repr = learner.rln(input_dict)
                output = fpln(_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                loss = learner.loss_fn(output, labels)
            loss = loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            output_softmax = F.softmax(output, -1)
            label_conf = output_softmax[np.arange(len(output_softmax)), labels] # Select labels in the softmax of 33 classes
            
            pred = models.utils.make_prediction(output.detach())
            
            all_losses.append(loss)
            all_predictions.extend(pred.tolist())
            all_labels.extend(labels.tolist())
            all_label_conf.extend(label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf

In [8]:
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf = evaluate(dataloader=test_dataloader, updates=updates, 
                                                                       mini_batch_size=mini_batch_size)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)

print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

----------Testing on test set starts here----------
Testing on YelpDataset
Support set metrics: Loss = 0.4177, accuracy = 0.8875, precision = 0.8988, recall = 0.8923, F1 score = 0.8913
Test metrics: Loss = 1.1012, accuracy = 0.5596, precision = 0.6217, recall = 0.5603, F1 score = 0.5693
Testing on AGNewsDataset
Support set metrics: Loss = 0.7551, accuracy = 0.7500, precision = 0.8318, recall = 0.8225, F1 score = 0.8036
Test metrics: Loss = 0.3922, accuracy = 0.8876, precision = 0.9004, recall = 0.8876, F1 score = 0.8935
Testing on DBPediaDataset
Support set metrics: Loss = 0.9310, accuracy = 0.7000, precision = 0.7356, recall = 0.7445, F1 score = 0.7290
Test metrics: Loss = 0.1178, accuracy = 0.9817, precision = 0.9832, recall = 0.9818, F1 score = 0.9824
Testing on AmazonDataset
Support set metrics: Loss = 0.6217, accuracy = 0.7875, precision = 0.8046, recall = 0.8228, F1 score = 0.8061
Test metrics: Loss = 1.1856, accuracy = 0.5507, precision = 0.6240, recall = 0.5510, F1 score = 0.56

In [9]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results_old.csv"
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-ori/OML-order1-id4-2023-04-25_22-37-58.874941_update5_results_old.csv
